## Blueprinty Case Study

### Introduction

Blueprinty is a small firm that makes software for developing blueprints specifically for submitting patent applications to the US patent office. Their marketing team would like to make the claim that patent applicants using Blueprinty's software are more successful in getting their patent applications approved. Ideal data to study such an effect might include the success rate of patent applications before using Blueprinty's software and after using it. unfortunately, such data is not available.

However, Blueprinty has collected data on 1,500 mature (non-startup) engineering firms. The data include each firm's number of patents awarded over the last 5 years, regional location, age since incorporation, and whether or not the firm uses Blueprinty's software. The marketing team would like to use this data to make the claim that firms using Blueprinty's software are more successful in getting their patent applications approved.

### Data

*todo: Read in data.*

In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import math 
from scipy.optimize import minimize
import patsy
from scipy.special import gammaln
data = pd.read_csv("blueprinty.csv")

In [ ]:
data.head()

In [ ]:
customer_yes = data[data['iscustomer']== 1]
customer_no = data[data['iscustomer']== 0]

In [ ]:
mean_customer_yes = customer_yes['patents'].mean()
mean_customer_no = customer_no['patents'].mean()

print(mean_customer_yes)
print(mean_customer_no)

*todo: Compare histograms and means of number of patents by customer status. What do you observe?*


In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 6))

# Plotting histograms for customer_yes and customer_no datasets
axs[0].hist(customer_yes['patents'], bins=10, color='blue', edgecolor='black')
axs[0].set_title('Customer Yes: Patents')
axs[0].set_xlabel('Patents')
axs[0].set_ylabel('Frequency')

axs[1].hist(customer_no['patents'], bins=10, color='orange', edgecolor='black')
axs[1].set_title('Customer No: Patents')
axs[1].set_xlabel('Patents')
axs[1].set_ylabel('Frequency')

# Adjust layout and show plots
plt.tight_layout()
plt.show()


Customer_no is right skewed compared to customer_yes, and from this we can confer that while the means show that customer_yes has a small number more patents than customer_no but customer_no has significantly more variance

Blueprinty customers are not selected at random. It may be important to account for systematic differences in the age and regional location of customers vs non-customers.




Customer no is right skewed compared to customer yes, and from this we can confer that while the means show that customer_yes has a small number more pattents than customer_no but customer_no has significantly more variance

Blueprinty customers are not selected at random. It may be important to account for systematic differences in the age and regional location of customers vs non-customers.

*todo: Compare regions and ages by customer status. What do you observe?*

## Customer Distribution by Region 

In [ ]:
# Grouping and summarizing for customer_yes DataFrame
grouped_yes = customer_yes.groupby('region').agg(
    count=('region', 'size'),
).reset_index()
grouped_yes['pct'] = grouped_yes['count'] / len(customer_yes)

# Grouping and summarizing for customer_no DataFrame
grouped_no = customer_no.groupby('region').agg(
    count=('region', 'size'),
).reset_index()
grouped_no['pct'] = grouped_no['count'] / len(customer_no)

# Viewing the summarized results
print(grouped_yes)
print(grouped_no)

The table above shows fewer customers in all regions in the customer_no category
The table splits customer data into two categories based on their status (Yes or No)
For customers with a "Yes" status:
Northeast has the highest percentage of customers at 57.36%.
The other regions follow much lower, with Southwest at 15.74%, South at 10.15%, Midwest at 8.63%, and Northwest at 8.12%.
For customers with a "No" status:
Northeast again leads with 37.45% of customers.
This is followed by Southwest at 20.41%, and Midwest, Northwest, and South all close to each other at 15.89%, 13.12%, and 13.12% respectively.

## Customer Distribution by Age

In [ ]:
# Calculate the mean age for customer_yes and customer_no groups
mean_age_yes = customer_yes['age'].mean()
mean_age_no = customer_no['age'].mean()

print("Mean Age of Customers Yes:", mean_age_yes)
print("Mean Age of Customers No:", mean_age_no)

# Create a 1-row x 2-column grid of subplots
fig, axs = plt.subplots(1, 2, figsize=(12, 6))

# Plot histogram of age for customer_yes
axs[0].hist(customer_yes['age'], bins=10, color='blue', edgecolor='black')
axs[0].set_title('Customer Yes: Age')
axs[0].set_xlabel('Age')
axs[0].set_ylabel('Frequency')

# Plot histogram of age for customer_no
axs[1].hist(customer_no['age'], bins=10, color='orange', edgecolor='black')
axs[1].set_title('Customer No: Age')
axs[1].set_xlabel('Age')
axs[1].set_ylabel('Frequency')

# Adjust layout and show the plots
plt.tight_layout()
plt.show()

### Observations
1- Region Impact: The Northeast dominates in customer numbers for both statuses, suggesting it may be a key market or has a larger base of the targeted demographic. 

2- Age Differences: Customers with a 'Yes' status are generally younger compared to those with a 'No' status, indicating that the younger customers are more active and engaged. 

3- Geographic Disparities: There is a clear geographic disparity in customer distribution. The Northeast has a much higher concentration of customers, possibly indicating regional preferences or more succesful market penetration. 

The likelihood (or log-likelihood) function (function of lambda and Y) for the Poisson model. 
### Estimation of Simple Poisson Model

Since our outcome variable of interest can only be small integer values per a set unit of time, we can use a Poisson density to model the number of patents awarded to each engineering firm over the last 5 years. We start by estimating a simple Poisson model via Maximum Likelihood.

*todo: Write down mathematically the likelihood for* $Y \sim \text{Poisson}(\lambda)$. Note that $f(Y|\lambda) = e^{-\lambda}\lambda^Y/Y!$.

$$
likelihood(f(Y|\lambda))=\Pi_{i=1}^ne^{-\lambda}\lambda^Y_i/Y_i!
$$

*todo: Code the likelihood (or log-likelihood) function for the Poisson model. This is a function of lambda and Y. For example:*


In [ ]:
def poisson_loglikelihood(lmbda, Y):
    # Convert Y to a NumPy array for easier numerical computations
    Y = np.array(Y)

    # Calculate the Poisson likelihood using NumPy's vectorized operations
    likelihood = np.exp(-lmbda) * np.power(lmbda, Y) / np.array([math.factorial(y) for y in Y])

    # Compute the sum of log-likelihoods
    ll = np.sum(np.log(likelihood))
    return ll
    ```



*todo: Use your function to plot lambda on the horizontal axis and the likelihood (or log-likelihood) on the vertical axis for a range of lambdas (use the observed number of patents as the input for Y).*





```{python}

*4 maximizes*

In [ ]:
def poisson_loglikelihood(lmbda, Y):
    Y = np.array(Y)
    likelihood = np.exp(-lmbda) * np.power(lmbda, Y) / np.array([np.math.factorial(y) for y in Y])
    ll = np.sum(np.log(likelihood))
    return ll

# Generate a sequence of lambda values from min to max of data['patents']
min_patents = data['patents'].min()
max_patents = data['patents'].max()
lambdas = np.arange(min_patents, max_patents + 1)

# Initialize an empty list to store log-likelihoods
lls = []

# Calculate the log-likelihood for each lambda value
for lmbda in lambdas:
    lls.append(poisson_loglikelihood(lmbda, data['patents']))

# Plot lambda values against the corresponding log-likelihoods
plt.plot(lambdas, lls, marker='o', linestyle='-')
plt.xlabel('Lambda')
plt.ylabel('Log-Likelihood')
plt.title('Poisson Log-Likelihood vs. Lambda')
plt.grid(True)
plt.show()

1- Peak Log-Likelihood- The log-likelihood reaches its peak around 𝜆 = 4 which suggests that this value is the best fit to the observed patent data per the Poisson model 

2- Decrease in Log-Likelihood- As 𝜆 increases beyond 4, there is a sharp decline in the log-likelihood indicating  that the higher values deviate from the best fit. 

3- Shape of the curve- The curve is unimodal with a single peak which is typical in likelihood analysis. This suggests a clear optimum value for 𝜆. 

*todo: If you're feeling mathematical, take the first derivative of your likelihood or log-likelihood, set it equal to zero and solve for lambda. You will find lambda_mle is Ybar, which "feels right" because the mean of a Poisson distribution is lambda.*

*todo: Find the MLE by optimizing your likelihood function with optim() in R or sp.optimize() in Python.*

### Optimizing the likelihood function to find the MLE 

In [ ]:
# Define the Poisson log-likelihood function (to maximize)
def poisson_loglikelihood(lmbda, Y):
    Y = np.array(Y)
    likelihood = np.exp(-lmbda) * np.power(lmbda, Y) / np.array([np.math.factorial(y) for y in Y])
    ll = np.sum(np.log(likelihood))
    return -ll  # Negate because we will minimize

# Sample dataset (replace this with your actual dataset)
Y = data['patents']

# Initial guess for lambda
initial_guess = 2

# Optimize using `minimize`, with negation to perform maximization
result = minimize(poisson_loglikelihood, initial_guess, args=(Y,), method='L-BFGS-B')

# The optimized lambda value (maximum likelihood estimate)
mle_lambda = result.x[0]

# Display the MLE for lambda
print(f"MLE for lambda: {mle_lambda}")

### Estimation of Poisson Regression Model

Next, we extend our simple Poisson model to a Poisson Regression Model such that $Y_i = \text{Poisson}(\lambda_i)$ where $\lambda_i = \exp(X_i'\beta)$. The interpretation is that the success rate of patent awards is not constant across all firms ($\lambda$) but rather is a function of firm characteristics $X_i$. Specifically, we will use the covariates age, age squared, region, and whether the firm is a customer of Blueprinty.

*todo: Update your likelihood or log-likelihood function with an additional argument to take in a covariate matrix X. Also change the parameter of the model from lambda to the beta vector. In this model, lambda must be a positive number, so we choose the inverse link function g() to be exp() so that* $\lambda_i = e^{X_i'\beta}$. *For example:*


In [ ]:
def poisson_regression_likelihood(beta, Y, X):
    # Convert inputs to NumPy arrays for efficient computation
    beta = np.array(beta)
    Y = np.array(Y)
    X = np.array(X)
    
    # Compute lambda as the linear combination of X and beta using matrix multiplication
    lambda_ = np.dot(X, beta)
    
    # Ensure lambda is positive to avoid issues with log and exponentiation
    lambda_ = np.exp(lambda_)

    # Calculate the Poisson log-likelihood terms
    ll = -lambda_ + Y * np.log(lambda_) - np.array([math.log(math.factorial(y)) for y in Y])
    
    # Sum up the log-likelihood values
    return np.sum(ll)

In [ ]:
def gr(beta, Y, X):
    # Convert inputs to NumPy arrays for efficient computation
    beta = np.array(beta)
    Y = np.array(Y)
    X = np.array(X)
    
    # Compute lambda (using matrix multiplication with exponential)
    lambda_ = np.dot(X, beta)
    lambda_ = np.exp(lambda_)
    
    # Calculate the gradient of the Poisson log-likelihood
    gradient = (Y / lambda_) - 1
    
    return gradient

*todo: Use your function along with R's optim() or Python's sp.optimize() to find the MLE vector and the Hessian of the Poisson model with covariates. Specifically, the first column of X should be all 1's to enable a constant term in the model, and the subsequent columns should be age, age squared, binary variables for all but one of the regions, and the binary customer variable. Use the Hessian to find standard errors of the beta parameter estimates and present a table of coefficients and standard errors.*

*todo: Check your results using R's glm() function or Python sm.GLM() function.*


In [ ]:
data['region_enc'] = data['region'].astype('category').cat.codes
X = data[['age', 'iscustomer', 'region_enc']].copy()
X['age2'] = data['age'] ** 2

y, X_design = patsy.dmatrices('patents ~ age + I(age **2) + iscustomer + region_enc', data=data, return_type='dataframe')

# Now, X_design is equivalent to the R model matrix, with the response variable in `y`

In [ ]:
# Scaling to 0->1
# for col in X.columns:
#     min = X[col].min()
#     max = X[col].max()
#     X[col] = (X[col]-min)/(max-min)

In [ ]:
# Ensure X is a numpy array
X = np.array(X)  # Replace with actual data loading if necessary

# Add an intercept column to X (column of ones at the beginning)
X_with_intercept = np.hstack([np.ones((X.shape[0], 1)), X])

# Define the likelihood function
def poisson_regression_likelihood(params, X, Y):
    # Ensure params is used directly as a NumPy array

    eta = np.dot(X, params)
    mu = np.exp(eta)
    log_likelihood = np.sum(-mu + Y * np.log(mu) - gammaln(Y + 1))
    return -log_likelihood

# Initial parameters guess, ensure it's a numpy array
initial_params = np.ones(X_with_intercept.shape[1]) * 0.1

# Run the minimization
result = minimize(poisson_regression_likelihood, initial_params, args=(X_with_intercept, Y), method='BFGS')
mle_beta = result.x

print("MLE Beta Parameters:", mle_beta)

*todo: Interpret the results. What do you conclude about the effect of Blueprinty's software on patent success?*

## AirBnB Case Study

### Introduction

AirBnB is a popular platform for booking short-term rentals. In March 2017, students Annika Awad, Evan Lebo, and Anna Linden scraped of 40,000 Airbnb listings from New York City. The data include the following variables:

::: {.callout-note collapse="true"}
### Variable Definitions

```         
- `id` = unique ID number for each unit
- `last_scraped` = date when information scraped
- `host_since` = date when host first listed the unit on Airbnb
- `days` = `last_scraped` - `host_since` = number of days the unit has been listed
- `room_type` = Entire home/apt., Private room, or Shared room
- `bathrooms` = number of bathrooms
- `bedrooms` = number of bedrooms
- `price` = price per night (dollars)
- `number_of_reviews` = number of reviews for the unit on Airbnb
- `review_scores_cleanliness` = a cleanliness score from reviews (1-10)
- `review_scores_location` = a "quality of location" score from reviews (1-10)
- `review_scores_value` = a "quality of value" score from reviews (1-10)
- `instant_bookable` = "t" if instantly bookable, "f" if not
```
:::

*todo: Assume the number of reviews is a good proxy for the number of bookings. Perform some exploratory data analysis to get a feel for the data, handle or drop observations with missing values on relevant variables, build one or more models (e.g., a poisson regression model for the number of bookings as proxied by the number of reviews), and interpret model coefficients to describe variation in the number of reviews as a function of the variables provided.*
